## Working with Quadkey in Vgrid DGGS

[![image](https://jupyterlite.rtfd.io/en/latest/_static/badge.svg)](https://demo.vgrid.vn/lab/index.html?path=vgrid/14_quadkey.ipynb)
[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeoshub/vgrid/blob/main/docs/notebooks/14_quadkey.ipynb)
[![image](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/opengeoshub/vgrid/HEAD?filepath=docs/notebooks/14_quadkey.ipynb)
[![image](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/opengeoshub/vgrid/blob/main/docs/notebooks/14_quadkey.ipynb)
[![image](https://jupyterlite.rtfd.io/en/latest/_static/badge.svg)](https://demo.gishub.vn/lab/index.html?path=notebooks/vgrid/14_quadkey.ipynb)

Full Vgrid DGGS documentation is available at [vgrid document](https://vgrid.gishub.vn).

To work with Vgrid DGGS directly in GeoPandas and Pandas, please use [vgridpandas](https://pypi.org/project/vgridpandas/). Full Vgridpandas DGGS documentation is available at [vgridpandas document](https://vgridpandas.gishub.vn).

To work with Vgrid DGGS in QGIS, install the [Vgrid Plugin](https://plugins.qgis.org/plugins/vgridtools/).

To visualize DGGS in Maplibre GL JS, try the [vgrid-maplibre](https://www.npmjs.com/package/vgrid-maplibre) library.

For an interactive demo, visit the [Vgrid Homepage](https://vgrid.vn).

### Install vgrid
Uncomment the following line to install [vgrid](https://pypi.org/project/vgrid/).

In [ ]:
# %pip install vgrid --upgrade

### latlon2quadkey

In [ ]:
from vgrid.conversion.latlon2dggs import latlon2quadkey

lat = 10.775276
lon = 106.706797
res = 14
quadkey_id = latlon2quadkey(lat, lon, res)
quadkey_id

### Quadkey to Shapely Polygon

In [ ]:
from vgrid.conversion.dggs2geo.quadkey2geo  import quadkey2geo

quadkey_geo = quadkey2geo(quadkey_id)
quadkey_geo

### Quadkey to GeoJSON

In [ ]:
from vgrid.conversion.dggs2geo.quadkey2geo import quadkey2geojson

quadkey_geojson = quadkey2geojson(quadkey_id)
quadkey_geojson

### Vector to Quadkey

In [ ]:
from vgrid.conversion.vector2dggs.vector2quadkey import vector2quadkey

file_path = (
    "https://raw.githubusercontent.com/opengeoshub/vopendata/main/shape/polygon.geojson"
)
vector_to_quadkey = vector2quadkey(file_path, resolution=18, compact=False, 
                        predicate = "intersects", output_format="gpd")
# Visualize the output
vector_to_quadkey.plot(edgecolor="white")

### Quadkey Compact

In [ ]:
from vgrid.conversion.dggscompact.quadkeycompact import quadkeycompact

quadkey_compacted = quadkeycompact(
    vector_to_quadkey, quadkey_id="quadkey", output_format="gpd"
)
quadkey_compacted.plot(edgecolor="white")

### Quadkey Expand

In [ ]:
from vgrid.conversion.dggscompact.quadkeycompact import quadkeyexpand

quadkey_expanded = quadkeyexpand(vector_to_quadkey, resolution=19, output_format="gpd")
quadkey_expanded.plot(edgecolor="white")

### Quadkey Binning

In [ ]:
from vgrid.binning.quadkeybin import quadkeybin
import geopandas as gpd
file_path = (
    "https://raw.githubusercontent.com/opengeoshub/vopendata/main/csv/dist1_pois.csv"
)
stats = "count"
quadkey_bin =  quadkeybin(file_path, resolution=18, stats=stats, 
                # numeric_field="confidence",
                # category="category",
                output_format="gpd")
quadkey_bin.plot(
    column=stats,        # numeric column to base the colors on
    cmap='Spectral_r',        # color scheme (matplotlib colormap)
    legend=True,  
    linewidth=0.2          # boundary width (optional)
)   

### Raster to Quadkey

In [ ]:
# %pip install folium

In [ ]:
from vgrid.conversion.raster2dggs.raster2quadkey    import raster2quadkey
from vgrid.utils.io import download_file          

raster_url = (
    "https://raw.githubusercontent.com/opengeoshub/vopendata/main/raster/rgb.tif"
)
raster_file = download_file(raster_url)

raster_to_quadkey =  raster2quadkey(raster_file,output_format="gpd")

# Visualize the output
import folium

m = folium.Map(tiles="CartoDB positron", max_zoom=28)

quadkey_layer = folium.GeoJson(
    raster_to_quadkey,
    style_function=lambda x: {
        "fillColor": f"rgb({x['properties']['band_1']}, {x['properties']['band_2']}, {x['properties']['band_3']})",
        "fillOpacity": 1,
        "color": "black",
        "weight": 1,
    },
    popup=folium.GeoJsonPopup(
        fields=["quadkey", "band_1", "band_2", "band_3"],
        aliases=["Quadkey ID", "Band 1", "Band 2", "Band 3"],
        style="""
            background-color: white;
            border: 2px solid black;
            border-radius: 3px;
            box-shadow: 3px;
        """,
    ),
).add_to(m)

m.fit_bounds(quadkey_layer.get_bounds())

# Display the map
m

### Quadkey Generator

In [ ]:
from vgrid.generator.quadkeygrid import quadkeygrid
# quadkey_grid = quadkeygrid(resolution=2,output_format="gpd")
quadkey_grid = quadkeygrid(resolution=18,bbox=[106.699007, 10.762811, 106.717674, 10.778649],output_format="gpd")
quadkey_grid.plot(edgecolor="white")

### Quadkey Inspect

In [ ]:
from vgrid.stats.quadkeystats import quadkeyinspect 
resolution = 7
quadkey_inspect = quadkeyinspect(resolution)    
quadkey_inspect.head()

### Distribution of Quadkey Area Distortions

In [ ]:
from vgrid.stats.quadkeystats import quadkey_norm_area
quadkey_norm_area(quadkey_inspect)

### Distribution of Quadkey IPQ Compactness

In [ ]:
from vgrid.stats.quadkeystats import quadkey_compactness
quadkey_compactness(quadkey_inspect)

### Quadkey Statistics

In [ ]:
from vgrid.stats import quadkeystats            
quadkeystats('km')